In [1]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
import numpy as np
import torch 
import numpy as np
import torch
from torch.optim import Adam
from model_static_SVI import static_SVI_CP, static_SVI_Tucker

import utils
# import data_loader
import time

In [13]:
data_file = '../processed_data/beijing_15k.npy'
full_data = np.load(data_file, allow_pickle=True).item()

fold=0
R_U = 3


# here should add one more data-loader class
data_dict = full_data['data'][fold]
data_dict['ndims'] = full_data['ndims']
data_dict['device'] = torch.device('cpu')
data_dict['v'] = 1
data_dict['v_time'] = 1


data_dict['U'] = [torch.rand(dim,R_U,1).double() for dim in data_dict['ndims']]

data_dict['R_U'] = 3
data_dict['gamma_size'] = 3

model = static_SVI_CP(data_dict)
optimizer = torch.optim.Adam(model.para_list, lr=1e-3)
batch_size = 512


for epoch in range(500):
    curr = 0
    while curr < model.N:
        batch_ind = np.random.choice(model.N, batch_size, replace=False)
        optimizer.zero_grad()
        loss = model.nELBO_batch(batch_ind)
        loss.backward(retain_graph=True)
        optimizer.step()
        curr = curr + batch_size

    if epoch % 2 == 0:
            loss_train,loss_test_rmse,loss_test_MAE = model.model_test(data_dict['te_ind'],torch.tensor(data_dict['te_y']))
            print('train loss %.4f, test loss %.4f'%(loss_train, loss_test_rmse))

train loss 1.0576, test loss 1.0176
train loss 1.0279, test loss 0.9868
train loss 1.0086, test loss 0.9674
train loss 0.9938, test loss 0.9538
train loss 0.9822, test loss 0.9431
train loss 0.9726, test loss 0.9350
train loss 0.9644, test loss 0.9285
train loss 0.9579, test loss 0.9233
train loss 0.9526, test loss 0.9191
train loss 0.9474, test loss 0.9143
train loss 0.9422, test loss 0.9094
train loss 0.9369, test loss 0.9038
train loss 0.9313, test loss 0.8989
train loss 0.9253, test loss 0.8928
train loss 0.9192, test loss 0.8859
train loss 0.9133, test loss 0.8796
train loss 0.9080, test loss 0.8740
train loss 0.9036, test loss 0.8691
train loss 0.9001, test loss 0.8650
train loss 0.8975, test loss 0.8618
train loss 0.8953, test loss 0.8599
train loss 0.8937, test loss 0.8585
train loss 0.8923, test loss 0.8574
train loss 0.8911, test loss 0.8560
train loss 0.8901, test loss 0.8549
train loss 0.8893, test loss 0.8538
train loss 0.8885, test loss 0.8534
train loss 0.8878, test loss

In [21]:
data_file = '../processed_data/beijing_15k.npy'
full_data = np.load(data_file, allow_pickle=True).item()

fold=0
R_U = 3


# here should add one more data-loader class
data_dict = full_data['data'][fold]
data_dict['ndims'] = full_data['ndims'] + [1461]
data_dict['device'] = torch.device('cpu')
data_dict['v'] = 1
data_dict['v_time'] = 1

data_dict['tr_ind'] = np.concatenate([data_dict['tr_ind'],data_dict['tr_T_disct'].reshape(-1,1)],1)
data_dict['te_ind'] = np.concatenate([data_dict['te_ind'],data_dict['te_T_disct'].reshape(-1,1)],1)

data_dict['U'] = [torch.rand(dim,R_U,1).double() for dim in data_dict['ndims']]

data_dict['R_U'] = 3
data_dict['gamma_size'] = 3

model = static_SVI_CP(data_dict)
optimizer = torch.optim.Adam(model.para_list, lr=1e-3)
batch_size = 512


for epoch in range(1000):
    curr = 0
    while curr < model.N:
        batch_ind = np.random.choice(model.N, batch_size, replace=False)
        optimizer.zero_grad()
        loss = model.nELBO_batch(batch_ind)
        loss.backward(retain_graph=True)
        optimizer.step()
        curr = curr + batch_size

    if epoch % 2 == 0:
            loss_train,loss_test_rmse,loss_test_MAE = model.model_test(data_dict['te_ind'],torch.tensor(data_dict['te_y']))
            print('train loss %.4f, test loss %.4f'%(loss_train, loss_test_rmse))

train loss 1.0035, test loss 0.9524
train loss 0.9892, test loss 0.9387
train loss 0.9740, test loss 0.9250
train loss 0.9560, test loss 0.9096
train loss 0.9348, test loss 0.8925
train loss 0.9117, test loss 0.8745
train loss 0.8875, test loss 0.8562
train loss 0.8614, test loss 0.8371
train loss 0.8321, test loss 0.8144
train loss 0.8000, test loss 0.7886
train loss 0.7645, test loss 0.7609
train loss 0.7291, test loss 0.7332
train loss 0.6963, test loss 0.7064
train loss 0.6635, test loss 0.6807
train loss 0.6315, test loss 0.6551
train loss 0.6012, test loss 0.6299
train loss 0.5710, test loss 0.6057
train loss 0.5431, test loss 0.5830
train loss 0.5146, test loss 0.5627
train loss 0.4876, test loss 0.5392
train loss 0.4623, test loss 0.5204
train loss 0.4385, test loss 0.5006
train loss 0.4162, test loss 0.4839
train loss 0.3955, test loss 0.4653
train loss 0.3752, test loss 0.4514
train loss 0.3569, test loss 0.4361
train loss 0.3404, test loss 0.4240
train loss 0.3252, test loss

In [22]:
loss_train,loss_test_rmse,loss_test_MAE = model.model_test(data_dict['te_ind'],torch.tensor(data_dict['te_y']))
print('train loss %.4f, test loss %.4f'%(loss_train, loss_test_rmse))

train loss 0.1361, test loss 0.2253


In [2]:
from model_static_CEP import static_CEP_CP



data_file = '../processed_data/beijing_15k.npy'
full_data = np.load(data_file, allow_pickle=True).item()

fold=0
R_U = 3


# here should add one more data-loader class
data_dict = full_data['data'][fold]
data_dict['ndims'] = full_data['ndims'] + [1461]
data_dict['device'] = torch.device('cpu')
data_dict['v'] = 1
data_dict['v_time'] = 1

data_dict['ind_tr'] = np.concatenate([data_dict['tr_ind'],data_dict['tr_T_disct'].reshape(-1,1)],1)
data_dict['ind_te'] = np.concatenate([data_dict['te_ind'],data_dict['te_T_disct'].reshape(-1,1)],1)

data_dict['y_tr'] = torch.tensor(data_dict['tr_y'])
data_dict['y_te'] = data_dict['te_y']


data_dict['N'] = len(data_dict['tr_ind'])
data_dict['U'] = [torch.rand(dim,R_U,1).double() for dim in data_dict['ndims']]

data_dict['R_U'] = 3
data_dict['gamma_size'] = 3

data_dict['a0'] = 1.0
data_dict['b0'] = 1.0

data_dict['DAMPPING_gamma']=0.1
data_dict['DAMPPING_U']=0.5

model = static_CEP_CP(data_dict)

RuntimeError: Expected 3-dimensional tensor, but got 4-dimensional tensor for argument #1 'batch1' (while checking arguments for bmm)